In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import from_levels_and_colors
import datetime

In [4]:
# Graficas por nivel de deslumbramiento, con los 12 sensores en una sola gráfica
#aulas = ['A10-1','A10-2','A20-1','A20-2','A40-1','A40-2']
aulas = ['A20-1','A20-2']

for aula in aulas:
    datos = pd.read_csv('../resultados/DGPu_{}_anual.ill'.format(aula), sep = " ", skiprows = range(0,10), header = None, skipinitialspace = True)
    DGPs = 6.22 * 10**-5 * datos + 0.184
    numsens, horas = DGPs.shape
    cmap, norm = from_levels_and_colors([0,1], ['red'])
    limits = np.array([[0.352,0.394],[0.395,0.456],[0.457,0.590]])
    titulo = 'Probabilidad de Deslumbramiento simplificada (DGPs) Aula {}'.format(aula)
    fig, ax = plt.subplots(figsize = (15,3))
    plt.title(titulo)
    ax.axvspan(0, 32, alpha=0.1, color='gray')
    ax.axvspan(331, 365, alpha=0.1, color='gray')
    ax.axvspan(151, 221, alpha=0.1, color='gray')
    colores = ['yellow','darkorange','crimson']
    for i in range(0,3):
        cmap, norm = from_levels_and_colors([0,1], [colores[i]])
        for sensor in range(0,12):
            LL = limits[i,0]
            UL = limits[i,1]
            DGPs_in = DGPs[DGPs > LL]
            DGPs_in = DGPs_in[DGPs_in < UL]
            plt.pcolor(DGPs_in.iloc[sensor].values.reshape(365,24).T,cmap = cmap, alpha = 0.2)
            plt.ylim(0,23)
            plt.xlim(0,365)
            plt.grid()

FileNotFoundError: [Errno 2] No such file or directory: '../resultados/DGPu_A20-1_anual.ill'

In [59]:
aulas = ['A10-1','A20-1','A40-1']
limits = np.array([[0.352,0.394],[0.395,0.456],[0.457,0.590]])
niveles = ['perceptible','molesto','insoportable']
for aula in aulas:
    datos = pd.read_csv('../resultados/DGPs/DGPu_{}_anual.ill'.format(aula), sep = " ", skiprows = range(0,10), header = None, skipinitialspace = True)
    DGPs = 6.22 * 10**-5 * datos + 0.184
    numsens, horas = DGPs.shape
    loc_sensores = pd.read_csv('../modelo/objects/DGPu_{}.pts'.format(aula), sep = " ",
                           header = None,names = ['x','y','z','dx','dy','dz'])
    for i in range(0,3):
        maximo_sensor = np.zeros([numsens,2])
        for sensor in range(0,numsens):
            LL = limits[i,0]
            UL = limits[i,1]
            DGPs_in = DGPs[DGPs > LL]
            DGPs_in = DGPs_in[DGPs_in < UL]
            maximo_sensor[sensor] = [int(DGPs_in.loc[sensor].argmax()),DGPs_in.loc[sensor].max()]
            
        np.savetxt('../resultados/DGPs/max_{}_{}.csv'.format(niveles[i],aula), maximo_sensor, delimiter = ",")
        

In [57]:
mx = pd.read_csv('../resultados/DGPs/max_perceptible_A10-1.csv', header = None)
mx.columns = ["posicion","valor"] 
mx.dropna(inplace = True)

In [58]:
mx

,0,1
0,7359.0,0.387945
1,543.0,0.388779
2,8488.0,0.393095
4,8342.0,0.352669
5,423.0,0.385383
6,7551.0,0.388013
7,591.0,0.392120
9,8246.0,0.361137
10,519.0,0.392039
11,87.0,0.393574


In [27]:
datos = pd.read_csv('../resultados/DGPu_A10-1_anual.ill', sep = " ", skiprows = range(0,10), header = None, skipinitialspace = True)
datos = datos.replace({0:np.nan, 0:np.nan}).transpose()
index = pd.date_range('2019-01-01 00:30:00', periods=8760, freq='H')
datos.index = index
DGPs = 6.22 * 10**-5 * datos + 0.184
Numsens = len(DGPs.columns)
#loc_sensores = pd.read_csv('../modelo/objects/DGPu_A10-1.pts', sep = " ",header = None,names = ['x','y','z','dx','dy','dz'])
#wea = pd.read_csv('../modelo/Cue.wea', header =None, skiprows=range(0,6),sep=" ")
#wea.columns=['mes','dia','hora','dir','dif']


In [3]:
h1 = '08:00:00' # Hora de inicio de jornada laboral 
h2 = '18:00:00' # Hora fin de jornada laboral
dif = 18 - 8
fi_1, ff_1 = '2019-01-31' , '2019-05-27' # Fechas de inicio y fin semestre 1
fi_2, ff_2 = '2019-08-01' , '2019-11-27' # Fechas de inicio y fin semestre 2 
DGPs = DGPs.between_time(h1,h2) # Selección de horario laboral
DGPs = pd.concat([DGPs[fi_1:ff_1],DGPs[fi_2:ff_2]]) # Selección de días de clases


In [3]:
# Arreglos para mantener una matriz de 8760 * Numsens
DGPs_anual = np.empty(8760)
DGPs_anual[:] = np.nan
index = pd.date_range('2019-01-01 00:30:00', periods=8760, freq='H')
DGPs_anual = pd.DataFrame(DGPs_anual, index = index)
DGPs_anual = pd.DataFrame(DGPs_anual.combine_first(DGPs))


NameError: name 'DGPs' is not defined